In [1]:
import psycopg2
import json
import csv
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values

In [3]:
def connect_to_postgres(dbname=None):
    """Connect to PostgreSQL. If dbname is None, connect to the default database."""
    return psycopg2.connect(
        dbname=dbname or "postgres",
        user="postgres",
        password="123456",
        host="localhost",
        port="5432"
    )

def create_database_if_not_exists(dbname):
    """Create the database if it does not exist."""
    conn = connect_to_postgres()
    conn.autocommit = True
    try:
        with conn.cursor() as cur:
            cur.execute(
                sql.SQL("CREATE DATABASE {};").format(sql.Identifier(dbname))
            )
            print(f"Database '{dbname}' created successfully.")
    except psycopg2.errors.DuplicateDatabase:
        print(f"Database '{dbname}' already exists.")
    finally:
        conn.close()

def create_tables_if_not_exists(conn):
    """Create tables in the database if they do not exist."""
    table_creation_queries = {
    "coffee_sales": """
        CREATE TABLE IF NOT EXISTS coffee_sales (
        transaction_id SERIAL PRIMARY KEY,
        date DATE NOT NULL,
        datetime TIMESTAMP NOT NULL,
        cash_type TEXT NOT NULL,
        card TEXT NOT NULL,
        money NUMERIC(10, 2) NOT NULL,
        coffee_name TEXT NOT NULL
    );
    """
    }
    
    with conn.cursor() as cur:
        for table, query in table_creation_queries.items():
            cur.execute(query)
        conn.commit()
    print("Tables created successfully (if they did not already exist).")

def insert_data_to_postgres(conn, table_name, data):
    """Insert data into a PostgreSQL table."""
    with conn.cursor() as cur:
        columns = data.columns.tolist()
        values = [tuple(x) for x in data.to_numpy()]
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(columns)}) 
        VALUES %s
        """
        execute_values(cur, insert_query, values)
        conn.commit()

In [4]:
# Define the database name
db_name = "coffee_sales"

# Create the database if it doesn't exist
create_database_if_not_exists(db_name)

# Connect to the newly created database
conn = connect_to_postgres(dbname=db_name)

# Ensure tables exist
create_tables_if_not_exists(conn)


Database 'coffee_sales' created successfully.
Tables created successfully (if they did not already exist).


In [6]:
coffee_sales_df = pd.read_csv('Coffee Sales.csv')

In [7]:
try:
    # Transfer data to PostgreSQL
    insert_data_to_postgres(conn, "coffee_sales", coffee_sales_df)
    print("Data successfully transferred to PostgreSQL database.")
finally:
    conn.close()

Data successfully transferred to PostgreSQL database.


In [10]:
coffee_sales_df

,date,datetime,cash_type,card,money,coffee_name
0,2024-03-01,2024-03-01 10:15:50.520,card,ANON-0000-0000-0001,38.70,Latte
1,2024-03-01,2024-03-01 12:19:22.539,card,ANON-0000-0000-0002,38.70,Hot Chocolate
2,2024-03-01,2024-03-01 12:20:18.089,card,ANON-0000-0000-0002,38.70,Hot Chocolate
3,2024-03-01,2024-03-01 13:46:33.006,card,ANON-0000-0000-0003,28.90,Americano
4,2024-03-01,2024-03-01 13:48:14.626,card,ANON-0000-0000-0004,38.70,Latte
...,...,...,...,...,...,...
2833,2025-01-22,2025-01-22 18:49:59.374,card,ANON-0000-0000-1119,35.76,Cocoa
2834,2025-01-22,2025-01-22 19:10:26.039,card,ANON-0000-0000-1051,30.86,Americano with Milk
2835,2025-01-22,2025-01-22 19:11:23.124,card,ANON-0000-0000-1051,35.76,Cappuccino
2836,2025-01-22,2025-01-22 19:29:15.703,card,ANON-0000-0000-1120,35.76,Latte
